# Q1. Running Elastic
Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:



curl localhost:9200

What's the version.build_hash value?

# 42f05b9372a9a4a470db3b52817899b99a76ee73

{
  "name" : "fc47f19a75f9",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Nf8oH9YgRti6QUmEkqp3Bw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },

# Getting the data
Now let's get the FAQ data. You can run this snippet:



In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': '4f71a0e27f36', 'cluster_name': 'docker-cluster', 'cluster_uuid': '3A0SdirXQOeXH20E0s1Hhg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
#Creating an index in Elastic similar

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index = index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

# Q2. Indexing the data

Which function do you use for adding your data to elastic?

insert 
## index ---- correct answer
put
add

In [5]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, 
                    document=doc)

100%|█████████████████████| 948/948 [00:24<00:00, 39.12it/s]


# Q3. Searching


Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

>*94.05
>*84.05
>* # 74.05 correct answer
>*64.05

Look at the _score field.

In [7]:
def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index = index_name, body=search_query)
    
    result_docs = []
    
    for hit in response ['hits']['hits']:
        result_docs.append(hit['_score'])
    
    
    return result_docs

In [8]:
query = "How do I execute a command in a running docker container?"
elastic_search(query)

[75.54128, 43.922554, 38.684105, 38.33403, 35.94081]

In [9]:

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index = index_name, body=search_query)

result_docs = []

for hit in response ['hits']['hits']:
    result_docs.append(hit['_score'])


result_docs

[75.54128, 43.922554, 38.684105, 38.33403, 35.94081]

In [10]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'FUSzO5ABhbz_olOMgQZf',
  '_score': 75.54128,
  '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
   '

# Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

>* How do I debug a docker container?
>* # How do I copy files from a different folder into docker container’s working directory? -- correct answer
>* How do Lambda container images work?

>* How can I annotate a graph?

In [11]:

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index = index_name, body=search_query)

result_docs = []

for hit in response ['hits']['hits']:
    result_docs.append(hit['_source'])


result_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [12]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

# Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

In [12]:
from openai import OpenAI

In [32]:
context_template = """
Q: {question}
A: {text}
""".strip()

# Build the context string
context_entries = [context_template.format(question=doc["question"], text=doc["text"]) for doc in result_docs]
context = "\n\n".join(context_entries)

len(context)
print(context)


Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

## Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:



In [14]:
query = "How do I execute a command in a running docker container?"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:{context}
""".strip()

# Construct the prompt
prompt = prompt_template.format(question=query, context=context)

print(prompt)
len(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/

1461

# What's the length of the resulting prompt? (use the len function)

>* 962
>* # 1462 correct answer
>* 1962
>* 2462

# Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

In [15]:
pip install tiktoken


Note: you may need to restart the kernel to use updated packages.


In [16]:
#Let's calculate the number of tokens in our query:
import tiktoken



In [17]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-4o")
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [29]:
num_tokens_from_string(prompt, "o200k_base")


322

## Use the encode function. How many tokens does our prompt have?

>* 122
>* 222
>* # 322 correct answer
>* 422

Note: to decode back a token into a word, you can use the decode_single_token_bytes function:

encoding.decode_single_token_bytes(63842)


# Bonus: calculating the costs (ungraded)
Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices here

On June 17, the prices for gpt4o are:

Input: $0.005 / 1K tokens
Output: $0.015 / 1K tokens
You can redo the calculations with the values you got in Q6 and Q7.

In [18]:
num_tokens_from_string(prompt, "o200k_base")


322

In [19]:
#Suppose that on average per request we send 150 tokens and receive back 250 tokens.

#How much will it cost to run 1000 requests?

total_tokens_sent =  1000 * 150
total_tokens_received =  1000 * 250

cost_sent = total_tokens_sent * (0.005/1000)
cost_received = total_tokens_received * (0.015/1000)

total_cost = cost_sent + cost_received

f"Total cost of 1000 requests is USD {total_cost:.2f}"


'Total cost of 1000 requests is USD 4.50'

# Bonus: generating the answer (ungraded)
Let's send the prompt to OpenAI. What's the response?

Note: you can replace OpenAI with Ollama. See module 2.

## Using Cohere free trial API

In [20]:
import cohere

In [21]:
co = cohere.Client()

In [38]:


# Preamble
preamble = "You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database."

# Split the context string into individual question-answer pairs
qa_pairs = context.split("\n\n")

# Create the documents list
documents = [{"title": f"chunk {idx}", "snippet": qa_pair.strip()} for idx, qa_pair in enumerate(qa_pairs)]

# Example of how the documents list looks like
documents

[{'title': 'chunk 0',
  'snippet': 'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)'},
 {'title': 'chunk 1',
  'snippet': "Q: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani"},
 {'title': 'chunk 2',
  'snippet': 'Q: How do I copy files from a different folder into docker container’s working di

In [44]:
#passing the query to cohere api
response = co.chat(
    message=query,
    documents=documents,
    preamble=preamble, 
    model="command-r",
)




In [45]:
print("Final answer:")
response.text

Final answer:


'To execute a command in a running Docker container, first find the container ID by running the command docker ps. Then, use the following command to execute a command in the specific container:\ndocker exec -it <container-id> bash'